# --- Settings ---

In [1]:
# Imports
import os, time, json, multiprocessing as mp
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import networkx as nx, osmnx as ox, umap
from node2vec import Node2Vec
from PIL import Image, ImageDraw, ImageFont
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="umap")

# Fast BLAS on Apple Silicon
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

# OSMnx settings
ox.settings.use_cache = True
ox.settings.log_console = False

# Layout params
thumb_size = (600, 600)
font_size, title_font_size = 20, 26
panel_width = 2 * thumb_size[0] + 3 * 40
panel_height = thumb_size[1] + 3 * font_size + 80

# Fonts
try:
    font = ImageFont.truetype("arial.ttf", font_size)
    title_font = ImageFont.truetype("arial.ttf", title_font_size)
except Exception:
    font = ImageFont.load_default()
    title_font = ImageFont.load_default()

# Files & directories
pdf_file = "comparison.pdf"
map_dir, tax_dir, net_dir, gra_dir = "maps", "taxonomy", "networks", "graphs"
for d in (map_dir, tax_dir, net_dir, gra_dir):
    os.makedirs(d, exist_ok=True)

city_data = [
    {"name":"Rome","country":"ITA","coordinates":(41.894096,12.485609),"distance":12000,"group":"Archetypal","taxonomy":"Radial_Implosion","network":"drive"},
    {"name":"Vatican_City","country":"VAT","coordinates":(41.902257,12.457421),"distance":200,"group":"Archetypal","taxonomy":"Elliptical_Implosion","network":"all"},
    {"name":"Fez","country":"MAR","coordinates":(34.065,-4.973),"distance":800,"group":"Archetypal","taxonomy":"Organic_Rhizome","network":"all"},
    {"name":"Moscow","country":"RUS","coordinates":(55.7558,37.6176),"distance":60000,"group":"Archetypal","taxonomy":"Centralized_Burst","network":"drive"},
    
    {"name":"Medellin","country":"COL","coordinates":(6.2518,-75.5836),"distance":15000,"group":"Geometrical","taxonomy":"Arc_Diagram","network":"all"},
    {"name":"Palmanova","country":"ITA","coordinates":(45.9061,13.3095),"distance":1500,"group":"Geometrical","taxonomy":"Radial_Convergence","network":"all"},
    {"name":"Dubai","country":"ARE","coordinates":(25.056530,55.207939),"distance":1000,"group":"Geometrical","taxonomy":"Segmented_Radial_Convergence","network":"all"},
    {"name":"Canberra","country":"AUS","coordinates":(-35.308188,149.124441),"distance":3200,"group":"Geometrical","taxonomy":"Centralized_Ring","network":"all"},
    
    {"name":"Los_Angeles","country":"USA","coordinates":(34.029315,-118.214444),"distance":800,"group":"Relational","taxonomy":"Flow_Chart","network":"drive"},
    {"name":"Randstad","country":"NLD","coordinates":(52.1,4.6),"distance":40000,"group":"Relational","taxonomy":"Area_Grouping","network":"drive"},
    {"name":"Greater_Cairo","country":"EGY","coordinates":(30.0444,31.2357),"distance":50000,"group":"Relational","taxonomy":"Circular_Ties","network":"drive"},
    {"name":"Amsterdam","country":"NLD","coordinates":(52.371,4.90),"distance":2000,"group":"Relational","taxonomy":"Ramification","network":"all"},
]

print(f"{len(city_data)} cities loaded")

12 cities loaded


# --- Graphs ---

In [45]:
for city in city_data:
    gpath = os.path.join(gra_dir, f"{city['name']}.graphml")

    if os.path.exists(gpath):
        print(f"🗂️  Graph exists — skipped: {city['name']}")
        continue

    try:
        print(f"🔄 {city['name']} ({city['network']}, r={city['distance']}m)…")
        G = ox.graph_from_point(city['coordinates'], dist=city['distance'], network_type=city['network'], simplify=True, retain_all=False)
        ox.save_graphml(G, gpath)
        print(f"✅ Saved: {gpath}")
        time.sleep(0.3)

    except Exception as e:
        print(f"⚠️ Failed for {city['name']}: {e}")

🗂️  Graph exists — skipped: Rome
🗂️  Graph exists — skipped: Vatican_City
🗂️  Graph exists — skipped: Fez
🔄 Moscow (drive, r=60000m)…
✅ Saved: graphs/Moscow.graphml
🔄 Medellin (all, r=15000m)…
✅ Saved: graphs/Medellin.graphml
🔄 Palmanova (all, r=1500m)…
✅ Saved: graphs/Palmanova.graphml
🔄 Dubai (all, r=1000m)…
✅ Saved: graphs/Dubai.graphml
🔄 Canberra (all, r=3200m)…
✅ Saved: graphs/Canberra.graphml
🔄 Los_Angeles (drive, r=800m)…
✅ Saved: graphs/Los_Angeles.graphml
🔄 Randstad (drive, r=40000m)…
✅ Saved: graphs/Randstad.graphml
🔄 Greater_Cairo (drive, r=50000m)…
✅ Saved: graphs/Greater_Cairo.graphml
🔄 Amsterdam (all, r=2000m)…
✅ Saved: graphs/Amsterdam.graphml


# --- Embeddings ---

In [ ]:
# ---- Embedding_and_Plotting ----
N2V_DIM, N2V_WALKLEN, N2V_NUMWALKS = 32, 15, 8
N2V_WINDOW, N2V_MINCOUNT, N2V_BATCHWORDS = 5, 1, 128
UMAP_NEIGHBORS, UMAP_MINDIST, UMAP_METRIC, UMAP_SEED = 15, 0.1, "cosine", 42

for city in city_data:
    gpath = os.path.join(gra_dir, f"{city['name']}.graphml")
    print(f"\n🏙️  Processing {city['name']}")

    # Load graph
    print("   - Loading graph…")
    G = ox.load_graphml(gpath)

    # Keep largest connected component
    print("   - Extracting component…")
    H = nx.Graph(G).subgraph(max(nx.connected_components(nx.Graph(G)), key=len)).copy()

    # Node2Vec
    print("   - Running Node2Vec…")
    n2v = Node2Vec(H, dimensions=N2V_DIM, walk_length=N2V_WALKLEN, num_walks=N2V_NUMWALKS,
                   p=1, q=1, workers=max(1, mp.cpu_count()-1), seed=42, quiet=True)
    model = n2v.fit(window=N2V_WINDOW, min_count=N2V_MINCOUNT, batch_words=N2V_BATCHWORDS)
    X = np.array([model.wv[str(n)] for n in H.nodes()])

    # UMAP projection
    print("   - Running UMAP…")
    embed = umap.UMAP(n_neighbors=UMAP_NEIGHBORS, min_dist=UMAP_MINDIST,
                      metric=UMAP_METRIC, random_state=UMAP_SEED).fit_transform(X)

    # Save figure
    out_path = os.path.join(net_dir, f"{city['name']}.png")
    plt.figure(figsize=(8, 8))
    plt.scatter(embed[:, 0], embed[:, 1], s=2, c="black", alpha=0.8)
    plt.axis("off")
    plt.savefig(out_path, dpi=300, bbox_inches="tight")
    plt.close()


🏙️  Processing Rome
   - Loading graph…
   - Extracting largest component…
   - Running Node2Vec…
   - Running UMAP…
💾 Saved image: networks/Rome_n2v_umap.png

🏙️  Processing Vatican_City
   - Loading graph…
   - Extracting largest component…
   - Running Node2Vec…
   - Running UMAP…
💾 Saved image: networks/Vatican_City_n2v_umap.png

🏙️  Processing Fez
   - Loading graph…
   - Extracting largest component…
   - Running Node2Vec…
   - Running UMAP…
💾 Saved image: networks/Fez_n2v_umap.png

🏙️  Processing Moscow
   - Loading graph…
   - Extracting largest component…
   - Running Node2Vec…
   - Running UMAP…
💾 Saved image: networks/Moscow_n2v_umap.png

🏙️  Processing Medellin
   - Loading graph…
   - Extracting largest component…
   - Running Node2Vec…
   - Running UMAP…
💾 Saved image: networks/Medellin_n2v_umap.png

🏙️  Processing Palmanova
   - Loading graph…
   - Extracting largest component…
   - Running Node2Vec…
   - Running UMAP…
💾 Saved image: networks/Palmanova_n2v_umap.png

🏙️ 

# --- Maps ---

In [3]:
# Use saved graphs to render map images (no downloads)

for city in city_data:
    print(f"– Generating map for {city['name']}…")
    gpath = os.path.join(gra_dir, f"{city['name']}.graphml")
    G = ox.load_graphml(gpath)

    # Project and draw
    G_proj = ox.project_graph(G)
    out_png = os.path.join(map_dir, f"{city['name']}.png")
    ox.plot_graph(G_proj, bgcolor="white", node_size=0, edge_color="black", edge_linewidth=0.3, show=False, save=True, filepath=out_png, dpi=300)
    plt.close("all")

– Generating map for Rome…
– Generating map for Vatican_City…
– Generating map for Fez…
– Generating map for Moscow…
– Generating map for Medellin…
– Generating map for Palmanova…
– Generating map for Dubai…
– Generating map for Canberra…
– Generating map for Los_Angeles…
– Generating map for Randstad…
– Generating map for Greater_Cairo…
– Generating map for Amsterdam…


In [12]:
# --- Panels ---

In [13]:
slides = []

for city in city_data:
    taxonomy_path = os.path.join(tax_dir, f"{city['taxonomy']}.jpg")
    city_path     = os.path.join(map_dir, f"{city['name']}.png")
    network_path  = os.path.join(net_dir, f"{city['name']}.png")

    taxonomy_img = Image.open(taxonomy_path).convert("RGB").resize(thumb_size)
    city_img     = Image.open(city_path).convert("RGB").resize(thumb_size)
    network_img  = Image.open(network_path).convert("RGB").resize(thumb_size)

    images = [taxonomy_img, city_img, network_img]

    # Auto panel size (3 images, equal margins)
    margin, y = 40, 100
    panel_width  = len(images) * thumb_size[0] + (len(images) + 1) * margin
    panel_height = thumb_size[1] + 200
    panel = Image.new("RGB", (panel_width, panel_height), "white")
    draw = ImageDraw.Draw(panel)

    # Paste images
    for i, img in enumerate(images):
        x = margin + i * (thumb_size[0] + margin)
        panel.paste(img, (x, y))

    # Title: name + taxonomy + coordinates + type + radius
    coords = f"({city['coordinates'][0]:.4f}, {city['coordinates'][1]:.4f})"
    title_text = f"{city['name']} — {city['taxonomy']} — {coords} - type={city['network']}, r={city['distance']} m"
    tw = draw.textlength(title_text, font=title_font) if hasattr(draw, "textlength") else title_font.getsize(title_text)[0]
    draw.text(((panel_width - tw) // 2, 20), title_text, font=title_font, fill="black")

    slides.append(panel)
    print(f"✅ Panel created: {city['name']}")

# Export to PDF (all slides)
comparison_images_rgb = [img.convert("RGB") for img in slides]
comparison_images_rgb[0].save(pdf_file, save_all=True, append_images=comparison_images_rgb[1:], format="PDF")
print(f"📄 Exported to: {pdf_file}")

✅ Panel created: Rome
✅ Panel created: Vatican_City
✅ Panel created: Fez
✅ Panel created: Moscow
✅ Panel created: Medellin
✅ Panel created: Palmanova
✅ Panel created: Dubai
✅ Panel created: Canberra
✅ Panel created: Los_Angeles
✅ Panel created: Randstad
✅ Panel created: Greater_Cairo
✅ Panel created: Amsterdam
📄 Exported to: comparison.pdf
